# 套件

In [1]:
import numpy as np
import pandas as pd
import jieba
import nltk
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from mlxtend.frequent_patterns import association_rules

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
jieba.load_userdict("dictionary.txt")

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/rk/fg780g753sd952335gvhq0bm0000gn/T/jieba.cache
Loading model cost 0.844 seconds.
Prefix dict has been built succesfully.


# 匯入資料

In [5]:
df_all = pd.read_csv("all_6data.csv",encoding='utf-8', usecols=['article','timestamp'])

In [67]:
fliter = df_all['timestamp'] <= 1551380000

In [70]:
df_all2 = df_all[fliter]

# 斷詞

In [7]:
def getText(data):
    sentence_train = list(data)
    return sentence_train

In [72]:
df_all_text = getText(df_all2['article'])

In [80]:
def cut_sentence(text):
    stop_list = [line[:-1] for line in open("stopwords.txt", encoding = 'utf8')]
    result = []
    for each in text :
        each_split=(jieba.cut(str(each)))
        each_result = [word for word in each_split if word not in stop_list and word  != ' ']
        result.append(' '.join(each_result))
    return result

In [81]:
df_all_cut = cut_sentence(df_all_text)

In [83]:
df_all_cut2 = [list(jieba.cut(sent)) for sent in df_all_cut]

In [84]:
stop_words =open("stopwords.txt", 'r').read()

In [85]:
df_all_cut3 = [[w for w in s if w not in stop_words] for s in df_all_cut2]

# Apriori

In [87]:
te = TransactionEncoder()
te_ary = te.fit_transform(df_all_cut3)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10238 entries, 0 to 10237
 to ﾟ
dtypes: bool(129103)
memory usage: 1.2 GB


In [58]:
frequent_itemsets = apriori(df, min_support=0.01,max_len=3, use_colnames=True)

# 第二次斷詞

In [ ]:
jieba.load_userdict("dictionary.txt")

In [ ]:
df_all_cut = cut_sentence(df_all_text)

In [ ]:
df_all_cut2 = [list(jieba.cut(sent)) for sent in df_all_cut]

In [ ]:
df_all_cut3 = [[w for w in s if w not in stop_words] for s in df_all_cut2]

# 第二次Apriori

In [ ]:
te = TransactionEncoder()
te_ary = te.fit_transform(df_all_cut)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.info()

In [ ]:
frequent_itemsets = apriori(df, min_support=0.01,max_len=1, use_colnames=True)

# 鎖定100個關鍵字

In [ ]:
key = data['欄位名'].values.tolist()

# One-hot

In [88]:
key = ['市場','經濟','黃金上漲','大盤上漲','大盤下跌','黃金下跌']

In [89]:
cut = [['市場','經濟','黃金上漲'],['經濟','黃金上漲','大盤上漲','大盤下跌']]

In [114]:
date01 = []
for k in cut:
    for i in key:
        if i not in k:
            date01.append(0)
        else:
            date01.append(1)

In [115]:
test01 = [date01[i:i+6] for i in range(0,len(date01),6)]

In [116]:
test01

[[1, 1, 1, 0, 0, 0], [0, 1, 1, 1, 1, 0]]

In [117]:
df = pd.DataFrame(test01)

In [119]:
df = df.T

In [120]:
df

,0,1
0,1,0
1,1,1
2,1,1
3,0,1
4,0,1
5,0,0


# 產生(x,y)資料集，y是下一個時間的關鍵字

In [ ]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

# 分割資料

In [ ]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

# 產生資料集

In [ ]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# LSTM建模

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)